# Transfer learning и Huggingface

Благодаря тому, что большие сети обладают хорошей обобщающей способностью, мы можем брать предобученные сети и использовать их в своих целях. Кроме того, сети можно дообучать на своих задачах.

Очень пополулярной библиотекой, хостингом для хранения и инференса (запуска) моделей является 🤗 **[Huggingface](https://huggingface.co/)**.

### mBART huggingface
* BART paper: https://arxiv.org/abs/1910.13461
* mBART paper: https://arxiv.org/abs/2001.08210
* Model: https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta

In [ ]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 32.6 MB/s 
     |████████████████████████████████| 1.3 MB 44.1 MB/s 
     |████████████████████████████████| 7.6 MB 53.6 MB/s 
     |████████████████████████████████| 182 kB 73.0 MB/s 


### Загружаем модели

In [ ]:
from transformers import AutoTokenizer, MBartForConditionalGeneration #AutoModel

model_name = "IlyaGusev/mbart_ru_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).cuda()

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.47G [00:00<?, ?B/s]

In [ ]:
print(type(tokenizer), type(model))

<class 'transformers.models.mbart.tokenization_mbart_fast.MBartTokenizerFast'> <class 'transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration'>


### Суммаризация текста

In [ ]:
article_text = """Колизей или амфитеатр Флавиев (лат. Amphitheatrum Flavium) — амфитеатр,
памятник архитектуры Древнего Рима, наиболее известное и одно из самых грандиозных сооружений Древнего мира, сохранившихся до нашего времени.
Находится в Риме, в низине между Эсквилинским, Палатинским и Целиевым холмами.
Строительство самого большого амфитеатра античного мира, вместимостью свыше 50 тыс. 
человек, велось на протяжении восьми лет как коллективное сооружение императоров династии Флавиев. 
Его начали строить в 72 году н. э. при императоре Веспасиане, а в 80 году н. э. амфитеатр был освящён императором Титом. 
Амфитеатр расположился на том месте, где был пруд, относившийся к Золотому дому Нерона.
История Колизея восходит к 68 году, когда измена преторианской гвардии и осуждение Сената 
заставили императора Нерона после четырнадцати лет деспотического управления государством покончить
с собой на загородной вилле неподалеку от Рима. Смерть Нерона привела к восемнадцатимесячной гражданской войне, 
закончившейся в 69 году. Победу в ней одержал Тит Флавий Веспасиан, которого в наши дни называют просто Веспасианом.
До того как стать императором, Веспасиан принимал участие в подавлении восстания иудеев, начавшегося в 66 году. 
После этого Веспасиан вместе с Титом в богатых восточных провинциях собирал налоги, чтобы привести в порядок 
государственные финансы, расстроенные Нероном и гражданской войной. Они возвратились в Рим в 71 году, чтобы отпраздновать победу над иудеями.
Став императором, Веспасиан решил реконструировать центр Рима и упрочить собственный культ, искоренив память о 
своём предшественнике Нероне. Оставалась нерешённой непростая проблема: что делать с дворцом Нерона, Золотым домом, 
как его называли, который вместе с прилегающим парком занимал в центре Рима площадь в 120 гектаров. 
Веспасиан решил разместить в нём имперские учреждения, а озеро возле дома засыпать и построить амфитеатр, 
предназначенный для развлечений народа. Это было хорошо взвешенное решение: постройкой амфитеатра земли, 
которыми пользовался Нерон, передавались народу.
"""

In [ ]:
input_ids = tokenizer.prepare_seq2seq_batch(
    [article_text],
    src_lang="ru_XX", 
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=600
)["input_ids"].cuda()

output_ids = model.generate(
    input_ids=input_ids,
    max_length=162,
    no_repeat_ngram_size=3,
    num_beams=5,
    top_k=0
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [ ]:
print(summary)

В Древнем Риме был построен самый большой амфитеатр античного мира, вместимостью свыше 50 тыс. человек. Его строительство велось на протяжении восьми лет как коллективное сооружение императоров династии Флавиев. Победу в ней одержал Тит Флавий Веспасиан.


In [ ]:
print(model)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN